In [1]:
import re
import json
import random
from collections import defaultdict
from tqdm import tqdm
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
import gc

In [17]:
# gc.collect()

In [2]:
df = pd.read_csv('habr_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4338 entries, 0 to 4337
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              4338 non-null   object
 1   title            4338 non-null   object
 2   theme            4338 non-null   object
 3   tags             4338 non-null   object
 4   read_time        4338 non-null   int64 
 5   number_of_views  4338 non-null   int64 
 6   date             4338 non-null   object
 7   amount_of_imgs   4338 non-null   int64 
 8   comments         4338 non-null   int64 
 9   bookmark         4338 non-null   int64 
 10  votes            4338 non-null   int64 
 11  text             4338 non-null   object
dtypes: int64(6), object(6)
memory usage: 406.8+ KB


In [12]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [19]:
re.sub(f"[{string.punctuation}«»—„“…\xa0]", " ", "text.text? Go_text")

'text text  Go text'

In [3]:
def remove_punctuation(text):
    return re.sub(f"[{string.punctuation}«»—„“…\xa0]", " ", text)

In [4]:
texts = df['text'].to_list()
texts = [remove_punctuation(i).lower() for i in texts]
len(texts)

4338

In [23]:
texts[:10]

['производство бриллиантов стало дешевле  чем когда либо  что создаёт странную проблему  слишком много бриллиантов в эпоху  когда в интернете можно купить практически всё  что угодно  нет ничего удивительного в том  что на китайском сайте электронной коммерции alibaba можно приобрести станок для изготовления бриллиантов за  200 000  если вы  как и я  не обращали внимания на алмазную промышленность  то окажется  что доступность этих машин отражает постоянную тенденцию к демократизации производства алмазов   процесс  начавшийся несколько десятилетий назад и продолжающий развиваться  изображение станка для производства синтетических алмазов hpht cubic press  изготовленного компанией henan huanghe whirlwind co   ltd  в китае история бриллиантов  выращенных в лаборатории  насчитывает не менее полувека  по словам аспиранта гарварда джавида лакха  javid lakha   написавшего обширную статью о выращенных в лаборатории алмазах  опубликованную в журнале works in progress в прошлом месяце  первый у

### Разбиение текста на токены

In [5]:
def tokenize_text(text):
    text = str(text).lower().strip()
    words = re.findall('[а-яёa-z]+', text)
    return ['<s>', '<s>'] + words + ['</s>']

In [26]:
print(tokenize_text(texts[0]))

['<s>', '<s>', 'производство', 'бриллиантов', 'стало', 'дешевле', 'чем', 'когда', 'либо', 'что', 'создаёт', 'странную', 'проблему', 'слишком', 'много', 'бриллиантов', 'в', 'эпоху', 'когда', 'в', 'интернете', 'можно', 'купить', 'практически', 'всё', 'что', 'угодно', 'нет', 'ничего', 'удивительного', 'в', 'том', 'что', 'на', 'китайском', 'сайте', 'электронной', 'коммерции', 'alibaba', 'можно', 'приобрести', 'станок', 'для', 'изготовления', 'бриллиантов', 'за', 'если', 'вы', 'как', 'и', 'я', 'не', 'обращали', 'внимания', 'на', 'алмазную', 'промышленность', 'то', 'окажется', 'что', 'доступность', 'этих', 'машин', 'отражает', 'постоянную', 'тенденцию', 'к', 'демократизации', 'производства', 'алмазов', 'процесс', 'начавшийся', 'несколько', 'десятилетий', 'назад', 'и', 'продолжающий', 'развиваться', 'изображение', 'станка', 'для', 'производства', 'синтетических', 'алмазов', 'hpht', 'cubic', 'press', 'изготовленного', 'компанией', 'henan', 'huanghe', 'whirlwind', 'co', 'ltd', 'в', 'китае', 'ис

In [6]:
tokenized_texts = list(map(tokenize_text, texts))
len(tokenized_texts)

4338

### Словарь из уникальных токенов с их количеством

In [7]:
tokens_freq = defaultdict(int)

for i in tokenized_texts:
    for token in i:
        tokens_freq[token] += 1

len(tokens_freq)

379189

In [34]:
list(tokens_freq.items())[:20]

[('<s>', 8676),
 ('производство', 996),
 ('бриллиантов', 5),
 ('стало', 1987),
 ('дешевле', 647),
 ('чем', 13738),
 ('когда', 14722),
 ('либо', 5023),
 ('что', 94127),
 ('создаёт', 395),
 ('странную', 24),
 ('проблему', 1081),
 ('слишком', 2405),
 ('много', 4773),
 ('в', 291655),
 ('эпоху', 363),
 ('интернете', 379),
 ('можно', 20510),
 ('купить', 363),
 ('практически', 2575)]

In [38]:
most_freq_tokens = sorted(list(tokens_freq.items()), key=lambda item: item[1], reverse=True)

In [41]:
print(most_freq_tokens[:100])

[('в', 291655), ('и', 277083), ('на', 137222), ('с', 104190), ('не', 95508), ('что', 94127), ('для', 63812), ('как', 56557), ('из', 53362), ('это', 53142), ('а', 51652), ('по', 49488), ('к', 44501), ('но', 43339), ('то', 40481), ('от', 39109), ('или', 31996), ('при', 27541), ('за', 27340), ('мы', 26266), ('так', 24952), ('если', 24018), ('у', 23767), ('его', 23339), ('о', 22809), ('до', 21914), ('он', 20985), ('их', 20788), ('можно', 20510), ('же', 19750), ('более', 19435), ('может', 18870), ('они', 18216), ('только', 18184), ('которые', 18015), ('чтобы', 17879), ('есть', 17841), ('все', 17757), ('было', 16619), ('я', 16143), ('когда', 14722), ('уже', 14518), ('будет', 14408), ('также', 14241), ('бы', 14124), ('чем', 13738), ('время', 13699), ('того', 12603), ('даже', 12465), ('году', 12383), ('во', 12282), ('был', 12062), ('том', 11892), ('очень', 11574), ('этом', 11309), ('который', 11259), ('быть', 11170), ('были', 11127), ('после', 10939), ('вы', 10482), ('этого', 10359), ('наприме

In [49]:
amount = 200
for i in most_freq_tokens:
    if not i[0] in stopwords.words('russian'):
        print(i, end=', ')
        amount -= 1
    if amount <= 0:
        break

('это', 53142), ('которые', 18015), ('также', 14241), ('время', 13699), ('году', 12383), ('очень', 11574), ('который', 11259), ('например', 10352), ('могут', 10004), ('всё', 9876), ('лет', 9648), ('года', 9411), ('ещё', 9197), ('<s>', 8676), ('просто', 7855), ('её', 7773), ('именно', 7689), ('является', 7676), ('однако', 7597), ('поэтому', 7593), ('времени', 7317), ('несколько', 7119), ('которая', 7007), ('системы', 6654), ('нужно', 6535), ('т', 5855), ('помощью', 5824), ('которых', 5797), ('данных', 5779), ('c', 5765), ('работы', 5655), ('нам', 5627), ('a', 5622), ('человека', 5387), ('случае', 5356), ('этих', 5294), ('хотя', 5291), ('часть', 5237), ('образом', 5236), ('других', 5127), ('м', 5053), ('d', 5029), ('либо', 5023), ('х', 4885), ('пока', 4848), ('энергии', 4746), ('людей', 4740), ('эта', 4697), ('такие', 4680), ('будут', 4660), ('жизни', 4658), ('лишь', 4620), ('количество', 4584), ('которой', 4550), ('модели', 4510), ('таких', 4502), ('человек', 4495), ('b', 4409), ('менее

### Создание 2-х словарей для преобразования токенов в уникальные номера и обратно

In [8]:
token2id = {'<e>': 0}
id_ = 1
for key, value in tokens_freq.items():
    token2id[key] = id_
    id_ += 1

In [56]:
len(token2id), token2id

(379190,
 {'<e>': 0,
  '<s>': 1,
  'производство': 2,
  'бриллиантов': 3,
  'стало': 4,
  'дешевле': 5,
  'чем': 6,
  'когда': 7,
  'либо': 8,
  'что': 9,
  'создаёт': 10,
  'странную': 11,
  'проблему': 12,
  'слишком': 13,
  'много': 14,
  'в': 15,
  'эпоху': 16,
  'интернете': 17,
  'можно': 18,
  'купить': 19,
  'практически': 20,
  'всё': 21,
  'угодно': 22,
  'нет': 23,
  'ничего': 24,
  'удивительного': 25,
  'том': 26,
  'на': 27,
  'китайском': 28,
  'сайте': 29,
  'электронной': 30,
  'коммерции': 31,
  'alibaba': 32,
  'приобрести': 33,
  'станок': 34,
  'для': 35,
  'изготовления': 36,
  'за': 37,
  'если': 38,
  'вы': 39,
  'как': 40,
  'и': 41,
  'я': 42,
  'не': 43,
  'обращали': 44,
  'внимания': 45,
  'алмазную': 46,
  'промышленность': 47,
  'то': 48,
  'окажется': 49,
  'доступность': 50,
  'этих': 51,
  'машин': 52,
  'отражает': 53,
  'постоянную': 54,
  'тенденцию': 55,
  'к': 56,
  'демократизации': 57,
  'производства': 58,
  'алмазов': 59,
  'процесс': 60,
  'н

In [9]:
id2token = {value: key for key, value in token2id.items()}

In [61]:
id2token

{0: '<e>',
 1: '<s>',
 2: 'производство',
 3: 'бриллиантов',
 4: 'стало',
 5: 'дешевле',
 6: 'чем',
 7: 'когда',
 8: 'либо',
 9: 'что',
 10: 'создаёт',
 11: 'странную',
 12: 'проблему',
 13: 'слишком',
 14: 'много',
 15: 'в',
 16: 'эпоху',
 17: 'интернете',
 18: 'можно',
 19: 'купить',
 20: 'практически',
 21: 'всё',
 22: 'угодно',
 23: 'нет',
 24: 'ничего',
 25: 'удивительного',
 26: 'том',
 27: 'на',
 28: 'китайском',
 29: 'сайте',
 30: 'электронной',
 31: 'коммерции',
 32: 'alibaba',
 33: 'приобрести',
 34: 'станок',
 35: 'для',
 36: 'изготовления',
 37: 'за',
 38: 'если',
 39: 'вы',
 40: 'как',
 41: 'и',
 42: 'я',
 43: 'не',
 44: 'обращали',
 45: 'внимания',
 46: 'алмазную',
 47: 'промышленность',
 48: 'то',
 49: 'окажется',
 50: 'доступность',
 51: 'этих',
 52: 'машин',
 53: 'отражает',
 54: 'постоянную',
 55: 'тенденцию',
 56: 'к',
 57: 'демократизации',
 58: 'производства',
 59: 'алмазов',
 60: 'процесс',
 61: 'начавшийся',
 62: 'несколько',
 63: 'десятилетий',
 64: 'назад',
 65

### Создание n-gram из текстов

In [10]:
def create_ngram(*args):
    ids = [str(token2id[i]) for i in args]
    return '_'.join(ids)

In [11]:
create_ngram('машин', 'отражает', 'постоянную', 'тенденцию')

'52_53_54_55'

In [12]:
LM_stat_trigram = defaultdict(lambda: defaultdict(lambda: 0))
LM_stat_bigram = defaultdict(lambda: 0)

In [13]:
for text in tqdm(tokenized_texts):
    for i in range(len(text)-2):
        t0 = text[i]
        t1 = text[i+1]
        t2 = text[i+2]
        code_2 = create_ngram(t0, t1)
        code_3 = create_ngram(t0, t1, t2)

        LM_stat_trigram[code_2][code_3] += 1
        LM_stat_bigram[code_2] += 1

100%|██████████████████████████████████████| 4338/4338 [00:23<00:00, 186.98it/s]


In [14]:
LM_stat_trigram

defaultdict(<function __main__.<lambda>()>,
            {'1_1': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'1_1_2': 1,
                          '1_1_15': 459,
                          '1_1_1507': 3,
                          '1_1_2128': 1,
                          '1_1_3751': 1,
                          '1_1_1606': 13,
                          '1_1_4856': 2,
                          '1_1_885': 129,
                          '1_1_3896': 1,
                          '1_1_5131': 3,
                          '1_1_7': 45,
                          '1_1_41': 11,
                          '1_1_10585': 1,
                          '1_1_2649': 2,
                          '1_1_148': 44,
                          '1_1_4073': 22,
                          '1_1_6342': 15,
                          '1_1_13450': 1,
                          '1_1_3282': 1,
                          '1_1_16239': 1,
                          '1_1_149': 9,
             

In [15]:
LM_stat_bigram

defaultdict(<function __main__.<lambda>()>,
            {'1_1': 4338,
             '1_2': 1,
             '2_3': 1,
             '3_4': 1,
             '4_5': 2,
             '5_6': 68,
             '6_7': 73,
             '7_8': 333,
             '8_9': 22,
             '9_10': 21,
             '10_11': 1,
             '11_12': 1,
             '12_13': 1,
             '13_14': 248,
             '14_3': 1,
             '3_15': 1,
             '15_16': 211,
             '16_7': 14,
             '7_15': 462,
             '15_17': 345,
             '17_18': 14,
             '18_19': 75,
             '19_20': 2,
             '20_21': 31,
             '21_9': 509,
             '9_22': 138,
             '22_23': 1,
             '23_24': 208,
             '24_25': 60,
             '25_15': 23,
             '15_26': 6748,
             '26_9': 5889,
             '9_27': 1204,
             '27_28': 17,
             '28_29': 3,
             '29_30': 1,
             '30_31': 9,
             '31_32

In [16]:
amount = 0
for k,v in LM_stat_trigram.items():
    amount += len(v)
amount

7374108

### Генерация текстов

In [18]:
def get_distribution(tok1, tok2):
    code = create_ngram(tok1, tok2)
    n = LM_stat_bigram[code]
    probs = []
    variants = []
    for k,v in LM_stat_trigram[code].items():
        if v > 0:
            variants.append(k.split('_')[-1])
            probs.append(v/n)
    return variants, probs

In [19]:
get_distribution('он', 'пошел')

(['15', '56', '87', '2522', '37'], [0.2, 0.2, 0.2, 0.2, 0.2])

In [20]:
def change_probs(probs: list, temperatue):
    logits = [np.log(p) for p in probs]
    new_logits = [l/temperatue for l in logits]
    new_probs = [np.exp(l) for l in new_logits]
    s = sum(new_probs)
    new_probs = [p/s for p in new_probs]
    return new_probs

In [21]:
probs = [0.5, 0.3, 0.15, 20]
change_probs(probs, 0.1)

[9.536743164062529e-17,
 5.766503906250007e-19,
 5.631351470947278e-22,
 0.9999999999999998]

In [25]:
np.random.multinomial(1, [0.8, 0.1, 0.05, 0.05])

array([1, 0, 0, 0])

In [27]:
def sample(variants, probs, temperature=None):
    if temperature is not None:
        probs = change_probs(probs, temperature)
    experiment = np.random.multinomial(1, probs)
    variants = np.array(variants).astype(int) * experiment
    for i in list(variants):
        if i > 0:
            return id2token[i]

In [23]:
def generate(tok1, tok2, temperature=None):
    res = [tok1, tok2]
    new_tok = None
    while new_tok != '</s>':
        variants, probs = get_distribution(tok1, tok2)
        new_tok = sample(variants, probs, temperature=temperature)
        res.append(new_tok)
        tok1 = tok2
        tok2 = new_tok
    return res

In [29]:
def gen_text(tok1, tok2, temperature=None):
    return ' '.join(generate(tok1, tok2, temperature))

In [46]:
len(LM_stat_bigram)

4187785

In [111]:
dict_iter = iter(LM_stat_bigram.keys())

In [116]:
rand_start = [random.randint(0, len(LM_stat_bigram)) for i in range(100)]

In [90]:
random.randint(0, 4000) 

1741

In [94]:
print(sorted(rand_start))

[13799, 106206, 223366, 323333, 354762, 411464, 462784, 564458, 565388, 613048, 615104, 615522, 636300, 649726, 664934, 691638, 765421, 766414, 794499, 800758, 866906, 1029725, 1039454, 1064427, 1066752, 1085661, 1104412, 1107555, 1169430, 1200108, 1271220, 1278708, 1332366, 1366543, 1370070, 1418756, 1419810, 1579887, 1683491, 1684198, 1757137, 1871508, 1885655, 1907289, 1910718, 1921309, 1953674, 1994821, 2006226, 2040082, 2048740, 2069403, 2107773, 2168235, 2192207, 2287141, 2291945, 2294777, 2316572, 2344595, 2389478, 2402346, 2514017, 2527334, 2568041, 2571159, 2574630, 2588093, 2662914, 2674364, 2684257, 2705466, 2738933, 2771277, 2772298, 2861426, 3032325, 3091044, 3091434, 3154071, 3243245, 3262348, 3275375, 3300575, 3359135, 3443358, 3510686, 3537699, 3559063, 3571513, 3696887, 3886900, 3929465, 3939057, 3947870, 4015324, 4060722, 4093298, 4095796, 4164993]


In [99]:
a = [1, 3, 4, 9]
a.pop(0), a

(1, [3, 4, 9])

In [100]:
a.pop(0), a

(3, [4, 9])

In [117]:
rand_start = sorted(rand_start)
key_list = []
for i in range(0, len(LM_stat_bigram)):
    start =  next(dict_iter)
    if len(rand_start) <= 0:
        break
    if i == rand_start[0]:
        key_list.append(start)
        rand_start.pop(0)
    i+=1

In [120]:
print(len(key_list), key_list)

100 ['15_2506', '20654_5266', '34141_1449', '27_3406', '44289_44290', '49971_21750', '3181_55310', '4505_61864', '120_62844', '67870_7357', '5794_2575', '86465_180', '43_62337', '84139_4406', '103362_4866', '1176_83295', '205_138180', '1412_1598', '37154_140965', '142198_142199', '145268_4515', '18010_41', '10821_415', '54751_604', '11357_543', '237_4871', '156613_14287', '68453_19961', '435_19307', '9651_15', '168_14999', '54375_62917', '175276_41', '38160_35', '1585_13240', '64777_193615', '6421_97863', '20964_2891', '8001_61691', '46900_152', '4524_36607', '26920_7255', '2390_255', '539_649', '217078_217079', '24407_64321', '224110_90856', '178167_856', '5586_3925', '235668_120175', '41153_7136', '53327_236823', '4959_7824', '10082_2762', '134_242808', '82277_184217', '56404_158936', '10525_1011', '7884_2451', '6689_16184', '15302_262509', '95310_11172', '70033_91618', '34751_8018', '17120_86980', '768_2246', '85453_496', '293759_3868', '221047_824', '15_69630', '31888_963', '947_10

In [131]:
def ids_to_text(ids):
    result = []
    for i in ids:
        tmp = []
        for j in i.split('_'):
            tmp.append(id2token[int(j)])
        result.append(tmp)
    return result

In [132]:
text_start = ids_to_text(key_list)
print(text_start)

[['в', 'computer'], ['сформировалось', 'зачем'], ['удалить', 'почти'], ['на', 'доступных'], ['поляризационную', 'плёнку'], ['keck', 'ii'], ['куда', 'сложней'], ['d', 'гермакрен'], ['из', 'колоды'], ['громоздких', 'устройств'], ['запрос', 't'], ['гелиосферы', 'под'], ['не', 'испытываете'], ['рисунку', 'a'], ['понеслось', 'вообще'], ['исчезновения', 'глыбы'], ['так', 'categoryпрям'], ['её', 'доказать'], ['японской', 'манге'], ['элегантная', 'вселеннаяджон'], ['насчитывающего', 'порядка'], ['вакцины', 'и'], ['эйнштейна', 'касается'], ['рубашки', 'которая'], ['атома', 'вокруг'], ['системы', 'означает'], ['сжижения', 'газа'], ['учат', 'одинаково'], ['бы', 'кучу'], ['банки', 'в'], ['от', 'activity'], ['телепортации', 'декогеренция'], ['детсадов', 'и'], ['переживаний', 'для'], ['научные', 'законы'], ['гостевой', 'командыteam'], ['полноценная', 'поисковая'], ['напряжение', 'постоянно'], ['трещины', 'вызывающие'], ['ключевого', 'метода'], ['значение', 'календаря'], ['активной', 'зоны'], ['сложн

### Генерация текстов

In [30]:
gen_text('я', 'пошел')

'я пошел на уступки новый контракт теперь же взлетели двое из этих случаев быть недо или пере измерение время ведёт международное расследование гибели людей один из его редакций а и в любом языке чуть ли не из приятных моментов на форумевсё это и многое другое если отойти от экономических спадов более того капитан является концентрированной версией целой команды поэтому к ним относятся европа ганимед и каллисто на момент окончания проекта млрд долларовтекущая стоимость млрд канал пересечёт европейскую часть континента в изложении использовать именно это осознание которое противоречит нашим представлениям а наполнена она нами переработанными мыслями некоторые представители отрасли предприняли небольшой но она не попадала сольвент это почти одновременно с увеличением угла изгиба работоспособность пневматического баллонного привода с тензодатчиком для микропальца была дополнительно обработана биосовместимым нанослоем золота d на e и l описывают наличие фотона в пространстве lab с хабраува

In [31]:
gen_text('я', 'пошел')

'я пошел на курс по data science предполагает хорошую статистическую подготовку навыки аналитического моделирования результаты исследования дж скотта армстронга которые профессон уортона провёл в другом месте не стоит очаровываться красивой картинкой скрывается опасная пустошь находящихся в особых задачах где была выполнена методом картирования который показал явное нарушение неравенств белла а остальным повторить работу последних двух слагаемых где хо rccc самый верхний слой наностолбиков неравномерного размера состоящий в том же сценарии кучи и дистрактора изображение на схеме оливера эванса френч родился в феврале го центр уже дважды запускала ещё менее успешными с точки зрения ведь скопление весьма велико градуса рассеянное звездное скопление разделить его в электрическую в солнечных масс и появившихся тогда когда один человек в живых клетках в большом магеллановом облаке бмо согласно оценкам в нашей крови находится в нескольких углеродных положениях a c данная версия ввн может раб

In [32]:
gen_text('я', 'пошел', temperature=0.3)

'я пошел по интересному пути биохакеров он в свою очередь может быть в курсе что ты не можешь не учить ребёнка слову мама и папа русские которые жили в компьютерной симуляции в ней не было бы здорово иметь такую редкую амплитуду яркости составляет примерно от до лет и это не так уж и много чего еще что то другое а именно в этот момент была убрана немцы посчитали что в свою очередь может быть не может быть и другие варианты например термотренировка когда микросхемы подвергаются только изгибу а не только в году в возрасте лет от земли до солнца до земли и луны и планет в солнечной системе и за счёт чего соседние слои в конечном итоге к началу х годов в сша в году в журнале nature communications doi s pmid s cid архивировано из оригинала на краях будут сбои фильтр нижних частот удаляет эти сбои и выставляют спавн десятка сильных противников в очень узком диапазоне температур от до лет в году в журнале nature communications чтобы вы понимали масштаб трагедии вот тут начинается самое интере

In [33]:
gen_text('я', 'пошел', temperature=0.9)

'я пошел на второй вентилятор называется силовой турбиной силовой потому что пионер космонавтики джеймс ван аллен направил его на землю мы не можем быть уверены лишь в процессе воспитания и к счастью с сообществе нашлись люди также промышляли рабовладением и эксплуатацией веб сервиса по бронированию слотов зарядных станций насчитывается ежегодный отчёт оон всемирное счастье сюда входит несколько категорий через одни он распространялся не причиняя им вреда возможные подходы включают ингибирование цитокинов и дать им тепло общения в том же направлении что если не покрыты толстым слоем в предыдущих исследованиях s ограничена менее чем за шесть часов после запуска кла приходит команда старт и разгон до км с скорости света что произойдёт если убрать процесс и результат прямо зависит от разницы в частоте колебаний с частотами выше полосы неустойчивости не растут не восстанавливаются с конца xvii века корни причин этого разрыва является смещение выборки в сотни тысяч лет перейдем к конкретике

In [36]:
gen_text('производство', 'бриллиантов', temperature=0.9)

'производство бриллиантов стало дешевле чем лечить но допустим в вашей повседневной активностью если какой то конкретный участок причём последовательно слева направо винтон серф роберт кан имевший огромный успешный опыт не может понять даже простейшие существа демонстрируют разные колебания в стальных пресс формах на заводе или вернуться туда откуда пришёл хорошо один глайдер удаляется из окна к этому моменту давно отбыл во францию бельгию нидерланды и норвегия находились в бедственном положении и местной межзвёздной среде наса рассматривает возможность инвестировать в управление двигателем стиральной машины устройство диагностика тахогенератор управление двигателем и превратить в прибыльный бизнес небесные тела движутся согласно строгим математическим законам которые включают в себя амплитуды вероятностей кубитов и обеспечить её рентабельность не получится нам нужно посчитать для куба в чистом виде а наша вселенная называется мультивселенной мы же появились отдельные персональные комп

In [37]:
gen_text('производство', 'бриллиантов', temperature=0.6)

'производство бриллиантов стало дешевле чем ее общая черта они распадаются на мелкие частицы затем микробы живущие в стрессе весь холестерин мозга синтезирован самим мозгом и телом передавались записывались и анализировались компьютером используя обнаруженные моторные нейронные сигналы чтобы выявить изменения признаки атрофии коры головного мозга содержит до отдельных аминокислот пищеварительными ферментами протеазами если вы делаете что то не очень хорошо работают как накопители кинетической энергии при гораздо более древние части головного и спинного мозга необязательно откладывать ради этого рузвельт готов был провести свои опыты но уже сейчас многие скажут да действительно опять вы смогли насколько я высокий воу я вообще ничего не значит что мы можем поговорить с внеземными цивилизациями вы не сможете встать с кресла один пользователь redit написал что ио к сожалению не было бы достаточно чтобы заменить целый декодерный механизм является самым позитивным образом копытные звери зайц

In [42]:
gen_text('бриллиантов', 'стало')

'бриллиантов стало дешевле чем стоимость энергии или любого онлайн сервиса компьютерная коммуникация меняет природу разговора и в отличие от четырёх человек в секунду с параметрами от числа попыток ввода например здесь и далее управлять перелетом к марсу и поясу астероидов esa одобрило проект венерианского зонда envision аппарат запустят к рейнеру гамма оснастят многоспектральным микроскопом для измерения его характеристик вполне достаточно чтобы мыши снова стали каплей обратив своё турбулентное распространение если условной гранью стим и киберпанке большое место в этих точках или около того самые массивные чёрные дыры росли так как во сне которые кажутся случайными и бессмысленными гэгами пародийные фильмы процветали в нашей солнечной системы после чего для того чтобы просто копировать модельку стеклянной стрелы в папку с файлами каталогами и метаданными которые являются единственными млекопитающими с меняющим цвет тапетумом и единственными структурами были облака газа и заставляют ег

In [134]:
gen_text_list = []
for i in text_start:
    gen_text_list.append(gen_text(*i))

In [137]:
df2 = pd.DataFrame(gen_text_list)
df2

,0
0,в computer science когда для доступа к логам м...
1,сформировалось зачем она вообще решаема ведь в...
2,удалить почти все молекулярные и т д методы уп...
3,на доступных знаниях или на неделю задержаться...
4,поляризационную плёнку для ещё одной не шча по...
...,...
95,цитотоксичность это свойство к левому шарику д...
96,сумм подписать контрольными суммами наш геноте...
97,уязвимы группам кроманьонских и неандертальски...
98,эвереттовской квантовой механике нет никаких д...


In [140]:
df2.to_csv('gen_statistic.csv')